<h2><center>Group 20</center></h2>
<h3><center>Jainit Patel - 201801172</center></h3>
<h3><center>Raj Desai - 201801183</center></h3>
<h3><center>Chirag Gupta - 201801188</center></h3>
<h3><center>Meet Parmar - 201801195</center></h3>
<h3><center>Raj Bhavsar - 201801437</center></h3>

# Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Speech Emotion Recognition - Feature Extraction



In [ ]:
# Import needed Libraries

import glob
import os
import librosa
import time
import numpy as np
import pandas as pd

### Load all files

We will create our numpy array extracting Mel-frequency cepstral coefficients (MFCCs) while the classes to predict will be extracted from the name of the file.

In [ ]:
# There are total 8 types of classes or emotions
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fear',
  '07':'disgust',
  '08':'surprised'
}

#defined tess emotions to test on TESS dataset only
tess_emotions=['angry','disgust','fear','ps','happy','sad']

##defined RAVDESS emotions to test on RAVDESS dataset only
ravdess_emotions=['neutral','calm','angry', 'happy','disgust','sad','fear','surprised']

observed_emotions = ['sad','angry','happy','disgust','surprised','neutral','calm','fear']

#### Feature extraction

Using librosa package we can extract the MFCC features. This function loads the file and after resampling and computing MFCC features, returns the features. We have selected the no. of MFCCs as 40.

In [ ]:
def extract_feature(file_name, mfcc):
    # Loading audio file with default sample rate of sr=22050
    X, sample_rate = librosa.load(os.path.join(file_name), res_type='kaiser_fast')
    result = []
    if mfcc:
        mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0) # Averaging mfcc for same central frequency of all samples for same audio
        result=np.hstack((result, mfccs)) 
    return result

#### Choosing a dataset

Choose the dataset(s) you want to load using the following function

In [ ]:
# Choose dataset ravdess and tess

def dataset_options():
    
    tess = True
    ravdess_speech = True
    ravdess_song = True
    data = {'ravdess':ravdess, 'ravdess_speech':ravdess_speech, 'ravdess_song':ravdess_song, 'tess':tess}
    print(data)
    return data

#### Load data

Load data from the datasets required which is obtained by calling the function dataset__options(). Extract features from each file with the selected emotions in chosen datasets using the extract_feature() function defined.

In [ ]:
# How to access data from GDrive

def load_data(test_size=0.2): 
    x,y=[],[]
    
    # feature to extract
    mfcc = True
    
    data = dataset_options()
    paths = []
    
    if data['ravdess_speech']:
        paths.append("drive/MyDrive/ML/REVDESS_SPEECH/Actor_*/*.wav")
    elif data['ravdess_song']:
        paths.append("drive/MyDrive/ML/REVDESS_SONG/Actor_*/*.wav")
        
    for path in paths:
        for file in glob.glob(path):
            file_name=os.path.basename(file)
            emotion=emotions[file_name.split("-")[2]]  #extract emotion from filename. dictionary emotions is defined above.
            if emotion not in observed_emotions:   #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    if data['tess']:
        for file in glob.glob("drive/MyDrive/ML/TESS/*AF_*.wav"):
            file_name=os.path.basename(file)
            emotion=file_name.split("_")[2][:-4] #split and remove .wav
            # As discussed above, we will take plesently surprised and surprised in single class
            if emotion == 'ps':
                emotion = 'surprised'
            if emotion not in observed_emotions: #options observed_emotions - RAVDESS and TESS, ravdess_emotions for RAVDESS only
                continue
            feature=extract_feature(file, mfcc)
            x.append(feature)
            y.append(emotion)
    return {"X":x,"y":y}

In [ ]:
start_time = time.time()

Trial_dict = load_data(test_size = 0.3)

print("--- Data loaded. Loading time: %s seconds ---" % (time.time() - start_time))

{'ravdess': False, 'ravdess_speech': True, 'ravdess_song': True, 'tess': True}
--- Data loaded. Loading time: 2463.3275349140167 seconds ---


In [ ]:
X = pd.DataFrame(Trial_dict["X"])
y = pd.DataFrame(Trial_dict["y"])

In [ ]:
X.shape, y.shape

((4240, 40), (4240, 1))

In [ ]:
#renaming the label column to emotion
y=y.rename(columns= {0: 'emotion'})

In [ ]:
#concatinating the attributes and label or emotion into a single dataframe
data = pd.concat([X, y], axis =1)

## Shuffling data

In [ ]:
#reindexing to shuffle the data at random
data = data.reindex(np.random.permutation(data.index))

In [ ]:
# Storing shuffled ravdess and tess data to avoid loading again
data.to_csv("drive/MyDrive/ML/RAVTESS_MFCC_Observed.csv")